In [1]:
import packages.caffemodel2pytorch as caffemodel2pytorch

In [3]:
import importlib
importlib.reload(caffemodel2pytorch)

<module 'packages.caffemodel2pytorch' from 'c:\\Users\\Admin\\Documents\\GitHub\\fundusfunfunf\\packages\\caffemodel2pytorch.py'>

In [4]:
import collections, torch, torchvision, h5py
import numpy as np

state_dict = h5py.File('./packages/converted.h5', 'r')

In [5]:
state_dict

<HDF5 file "converted.h5" (mode r)>

In [15]:
from deepdraw.models.driu import driu
model = driu()

In [18]:
model

Sequential(
  (normalizer): TorchVisionNormalizer()
  (backbone): VGG4Segmentation(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), 

In [14]:
for k, v in state_dict.items():
    print(k)
    print(np.array(v).shape)

conv1_1.bias
(64,)
conv1_1.weight
(64, 3, 3, 3)
conv1_2.bias
(64,)
conv1_2.weight
(64, 64, 3, 3)
conv1_2_16.bias
(16,)
conv1_2_16.weight
(16, 64, 3, 3)
conv2_1.bias
(128,)
conv2_1.weight
(128, 64, 3, 3)
conv2_2.bias
(128,)
conv2_2.weight
(128, 128, 3, 3)
conv2_2_16.bias
(16,)
conv2_2_16.weight
(16, 128, 3, 3)
conv3_1.bias
(256,)
conv3_1.weight
(256, 128, 3, 3)
conv3_2.bias
(256,)
conv3_2.weight
(256, 256, 3, 3)
conv3_3.bias
(256,)
conv3_3.weight
(256, 256, 3, 3)
conv3_3_16.bias
(16,)
conv3_3_16.weight
(16, 256, 3, 3)
conv4_1.bias
(512,)
conv4_1.weight
(512, 256, 3, 3)
conv4_2.bias
(512,)
conv4_2.weight
(512, 512, 3, 3)
conv4_3.bias
(512,)
conv4_3.weight
(512, 512, 3, 3)
conv4_3_16.bias
(16,)
conv4_3_16.weight
(16, 512, 3, 3)
new-score-weighting.bias
(1,)
new-score-weighting.weight
(1, 64, 1, 1)
score-dsn_1.bias
(1,)
score-dsn_1.weight
(1, 16, 1, 1)
score-dsn_2.bias
(1,)
score-dsn_2.weight
(1, 16, 1, 1)
score-dsn_3.bias
(1,)
score-dsn_3.weight
(1, 16, 1, 1)
score-dsn_4.bias
(1,)
score-d

In [6]:
import google.protobuf
caffe_proto = 'https://raw.githubusercontent.com/BVLC/caffe/master/src/caffe/proto/caffe.proto'
net_param = caffemodel2pytorch.initialize(caffe_proto).NetParameter()
prototxt = 'packages/deploy_DRIVE.prototxt'
google.protobuf.text_format.Parse(open(prototxt).read(), net_param)

name: "DRIU_DRIVE"
input: "data"
input_dim: 1
input_dim: 3
input_dim: 584
input_dim: 565
layer {
  name: "conv1_1"
  type: "Convolution"
  bottom: "data"
  top: "conv1_1"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 64
    pad: 1
    kernel_size: 3
    engine: CAFFE
  }
}
layer {
  name: "relu1_1"
  type: "ReLU"
  bottom: "conv1_1"
  top: "conv1_1"
}
layer {
  name: "conv1_2"
  type: "Convolution"
  bottom: "conv1_1"
  top: "conv1_2"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 64
    pad: 1
    kernel_size: 3
    engine: CAFFE
  }
}
layer {
  name: "relu1_2"
  type: "ReLU"
  bottom: "conv1_2"
  top: "conv1_2"
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1_2"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 2
    stride: 2
  }
}
layer {
  name: "conv2_1"
  type: "Convolu

In [23]:
import torch

model = caffemodel2pytorch.Net(
	prototxt = 'packages/deploy_DRIVE.prototxt',
	weights = 'packages/DRIU_DRIVE.caffemodel',
	caffe_proto = 'https://raw.githubusercontent.com/BVLC/caffe/master/src/caffe/proto/caffe.proto'
)
model.cuda()
model.eval()
torch.set_grad_enabled(False)

# make sure to have right procedure of image normalization and channel reordering
image = torch.Tensor(8, 3, 224, 224).cuda()

# outputs dict of PyTorch Variables
# in this example the dict contains the only key "prob"
#output_dict = model(data = image)

# a single input variable is interpreted as an input blob named "data"
# in this example the dict contains the only key "fc7"
output_dict = model(image)

Skipping layer [upsample2_, Deconvolution, Deconvolution]: not found in caffemodel2pytorch.modules dict
Skipping layer [crop, Crop, Crop]: not found in caffemodel2pytorch.modules dict
Skipping layer [upsample4_, Deconvolution, Deconvolution]: not found in caffemodel2pytorch.modules dict
Skipping layer [crop, Crop, Crop]: not found in caffemodel2pytorch.modules dict
Skipping layer [upsample8_, Deconvolution, Deconvolution]: not found in caffemodel2pytorch.modules dict
Skipping layer [crop, Crop, Crop]: not found in caffemodel2pytorch.modules dict
Skipping layer [concat, Concat, Concat]: not found in caffemodel2pytorch.modules dict
Skipping layer [sigmoid-fuse, Sigmoid, Sigmoid]: not found in caffemodel2pytorch.modules dict
caffemodel2pytorch: loading model from [packages/DRIU_DRIVE.caffemodel] in HDF5 format failed [Unable to open file (file signature not found)], falling back to caffemodel format
caffemodel2pytorch: loaded model from [packages/DRIU_DRIVE.caffemodel] in caffemodel forma

AssertionError: Variable [concat-upscore] does not exist. Pass it as a keyword argument or provide a layer which produces it.